In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder



llm = ChatOpenAI(
    temperature=0.1,
)

loader = TextLoader("./document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=200,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful document information extractor. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        (
            "human",
            "{question}"
        ),
        MessagesPlaceholder(variable_name="history"),

    ]
)

memory = ConversationBufferMemory(
    llm = llm,
    return_messages= True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
        "history" : load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [73]:
invoke_chain("Is Aaronson guilty?")

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 2174, 'total_tokens': 2198, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ce558548-a5a9-4ee5-8481-c45c607e0cf3-0'


In [74]:
invoke_chain("What message did he write in the table?")

content="I don't know." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 2208, 'total_tokens': 2214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3edafbfa-fdae-425e-a58f-dd79b676bb06-0'


In [75]:
invoke_chain("Who is Julia?")

content="I don't know." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 2229, 'total_tokens': 2235, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-49121410-1f46-402a-9cf7-99cc2260d062-0'
